<a href="https://colab.research.google.com/github/Joshalphonse/BMF-video-similarity-search/blob/main/Video_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a notebook dedicated to learn how to create a video similarity search with BMF and Pinecone(Vector Database). Make sure to add your own .env file, and create the necessary folders to use this. For a more indepth breakfown please check out the blog

In [ ]:
# Import and configure logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create a handler to log to stdout
handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(handler)

In [ ]:
!pip install -qU \
  pinecone-client \
  BabitMF-GPU \
  torch \
  torchvision>=0.12.0 \
  python-dotenv \
  av

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.2.2 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.2.2 which is incompatible.


In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
import sys
_=load_dotenv(find_dotenv()) # read local .env file

Myvar = os.environ['PINECONE_API_KEY']
print(Myvar)

e879cc7b-6aa6-4f72-bfcd-dcf59b342196


In [ ]:
! curl -L https://github.com/towhee-io/examples/releases/download/data/reverse_video_search.zip -O
! unzip -q -o reverse_video_search.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  152M  100  152M    0     0  99.6M      0  0:00:01  0:00:01 --:--:--  209M


In [ ]:
import pandas as pd

df = pd.read_csv('./reverse_video_search.csv', nrows=3) #put the files in the dataframe
video_paths = df['path'].tolist() #convert df to python list

print(video_paths) #check if the video paths

['./train/country_line_dancing/bTbC3w_NIvM.mp4', './train/country_line_dancing/n2dWtEmNn5c.mp4', './train/country_line_dancing/zta-Iv-xK7I.mp4']


In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models

model = models.resnet18(pretrained=True)
model.eval()

print(model.eval())

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 171MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
import os
import cv2
import av
from pinecone import Pinecone, ServerlessSpec
import numpy as np
import pandas as pd
import torch
import torchvision.transforms as transforms
import torchvision.models as models


PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_ENVIRONMENT = os.environ["PINECONE_ENVIRONMENT"]
PINECONE_DATABASE = os.environ["PINECONE_DATABASE"]

# Replace 'your_pinecone_api_key' with your actual Pinecone API key or use environment variables like I am here
pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)
index = pc.Index(PINECONE_DATABASE)

#load the CSV file
csv_file = './reverse_video_search.csv'
df = pd.read_csv(csv_file, nrows=3)
video_paths = df['path'].tolist()
print(video_paths) #check if the video paths

#load a pretrained ResNet model
model = models.resnet18(pretrained = True)
model.eval()

#remove the last fully connectected layer
model = torch.nn.Sequential(*list(model.children())[:-1])

# Define the preprocessing transforms
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Iterate over each video path and generate embeddings
for video_path in video_paths:
    # Open the video file
    video = av.open(video_path)

    # Get the first video stream
    video_stream = next(s for s in video.streams if s.type == 'video')

    # Initialize variables for storing embeddings
    embeddings = []

    # Iterate over the video frames
    for frame in video.decode(video=0):
        # Convert the frame to a numpy array
        img = frame.to_ndarray(format='rgb24')

        # Preprocess the frame
        img = preprocess(img)
        img = img.unsqueeze(0)  # Add batch dimension

        # Generate embeddings using the ResNet model
        with torch.no_grad():
            embedding = model(img)
            embedding = embedding.squeeze().numpy()

        # Append the embedding to the list
        embeddings.append(embedding)

    # Convert the list of embeddings to a numpy array
    embeddings = np.array(embeddings)

    # Calculate the average embedding for the video
    avg_embedding = np.mean(embeddings, axis=0)
    print(avg_embedding)

    # Upsert the embedding to Pinecone
    index.upsert(
        vectors=[
            (video_path, avg_embedding.tolist())
        ],
        namespace='video_embeddings'
    )

    print(f"Upserted embedding for video: {video_path}")

['./train/country_line_dancing/bTbC3w_NIvM.mp4', './train/country_line_dancing/n2dWtEmNn5c.mp4', './train/country_line_dancing/zta-Iv-xK7I.mp4']


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[0.1975852  2.749517   0.84790474 1.0632831  0.25560564 0.30372965
 0.10311005 0.6781837  0.30272514 0.67747945 0.5228789  0.733537
 0.584947   0.6034906  0.35398906 0.5630128  1.7083528  0.7473611
 0.28747582 0.6528601  0.25462946 1.4609638  0.8950826  2.1040838
 0.62127596 0.22613691 1.7954069  1.331547   0.33952639 1.1530455
 0.8457775  0.93469185 0.32220775 0.80065143 0.961425   0.36470413
 0.8630163  1.0693367  0.41804913 2.0176532  0.1717425  0.77546763
 1.8746331  0.8184808  0.90874606 0.322206   0.3044036  0.81794137
 0.80901265 0.51200134 0.92606044 0.7719966  0.5612946  0.39628744
 0.30152896 1.4736904  0.901579   1.0201709  1.2198595  0.36913177
 0.4206543  0.49619627 1.4402015  0.6565788  0.8162852  1.2335038
 1.0062577  0.35270593 1.5679954  0.6802399  1.8659592  0.6454067
 1.5738058  0.14128596 1.5759013  0.33664018 1.0016316  1.0551246
 0.6964168  0.48590976 0.68266565 0.9828467  0.2236214  2.876266
 0.4438299  0.30294994 0.42758146 0.7031935  2.9979553  1.1821252
 0.214

In [ ]:
!ls /usr/local/lib/python3.10/dist-packages/bmf/lib/

_bmf.cpython-310-x86_64-linux-gnu.so  libbmf_module_sdk.so.0	   libengine.so.0
_hmp.cpython-310-x86_64-linux-gnu.so  libbmf_module_sdk.so.0.0.9   libengine.so.0.0.9
libbackward.a			      libbmf_py_loader.so	   libfmt.a
libbenchmark.a			      libbuiltin_modules.so	   libhmp.so
libbenchmark_main.a		      libbuiltin_modules.so.0	   libhmp.so.1
libbmf_go_loader.so		      libbuiltin_modules.so.0.0.9  libhmp.so.1.2.0
libbmf_module_sdk.so		      libengine.so		   libspdlog.a


In [ ]:
%env LIBRARY_PATH=$LIBRARY_PATH:/usr/local/lib/python3.10/dist-packages/bmf/lib

env: LIBRARY_PATH=$LIBRARY_PATH:/usr/local/lib/python3.10/dist-packages/bmf/lib


In [ ]:
!sudo apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!sudo apt install libdw1

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libdw1 is already the newest version (0.186-1build1).
libdw1 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!dpkg -l | grep -i ffmpeg

ii  ffmpeg                                    7:4.4.2-0ubuntu0.22.04.1                amd64        Tools for transcoding, streaming and playing of multimedia files
ii  libavcodec-dev:amd64                      7:4.4.2-0ubuntu0.22.04.1                amd64        FFmpeg library with de/encoders for audio/video codecs - development files
ii  libavcodec58:amd64                        7:4.4.2-0ubuntu0.22.04.1                amd64        FFmpeg library with de/encoders for audio/video codecs - runtime files
ii  libavdevice58:amd64                       7:4.4.2-0ubuntu0.22.04.1                amd64        FFmpeg library for handling input and output devices - runtime files
ii  libavfilter7:amd64                        7:4.4.2-0ubuntu0.22.04.1                amd64        FFmpeg library containing media filters - runtime files
ii  libavformat-dev:amd64                     7:4.4.2-0ubuntu0.22.04.1                amd64        FFmpeg library with (de)muxers for multimedia containers - development

In [ ]:
!ffmpeg -version

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

In [ ]:
!pip install wurlitzer
%load_ext wurlitzer

In [ ]:
import os
import glob
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from pinecone import Pinecone
import bmf
import cv2

PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_ENVIRONMENT = os.environ["PINECONE_ENVIRONMENT"]
PINECONE_DATABASE = os.environ["PINECONE_DATABASE"]

# Replace 'your_pinecone_api_key' with your actual Pinecone API key or use environment variables like I am here
pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)
index = pc.Index(PINECONE_DATABASE)

model = models.resnet18(pretrained=True)
model.eval()
#remove the last fully connectected layer
model = torch.nn.Sequential(*list(model.children())[:-1])

preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

input_video_path = "/content/linedancing.mp4"
output_path = "./extracted-images/simple_%03d.jpg"

graph = bmf.graph({'dump_graph':1})
video = graph.decode({
    "input_path": input_video_path,
}).fps(2)

try:
    (
        bmf.encode(
            video['video'],
            None,
            {
                "output_path": output_path,
                "format": "image2",
                "video_params": {"codec": "jpg"},
            }
        ).run()
    )
    print("Frame extraction completed successfully.")
except Exception as e:
    print(f"Error during frame extraction: {str(e)}")
# Load the extracted query frames and generate embeddings
query_frame_paths = glob.glob(output_path.replace("%03d", "*"))
query_embeddings = []

for frame_path in query_frame_paths:
    container = av.open(frame_path)
    frame = next(container.decode(video=0))

    img = frame.to_ndarray(format='rgb24')
    img = preprocess(img)
    img = img.unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        embedding = model(img)
        embedding = embedding.squeeze().numpy()

    query_embeddings.append(embedding)

query_embeddings = np.array(query_embeddings)
avg_query_embedding = np.mean(query_embeddings, axis=0)

# Perform similarity search using Pinecone
num_results = 5  # Number of similar videos to retrieve
results = index.query(
    vector=avg_query_embedding.tolist(),
    top_k=num_results,
    include_metadata=True,
    namespace='video_embeddings'
)

# Print the most similar video paths
for match in results['matches']:
    video_path = match['id']
    print(f"Similar video: {video_path}")

{
    "input_streams": [],
    "output_streams": [],
    "nodes": [
        {
            "module_info": {
                "name": "c_ffmpeg_decoder",
                "type": "",
                "path": "",
                "entry": ""
            },
            "meta_info": {
                "premodule_id": -1,
                "callback_binding": []
            },
            "option": {
                "input_path": "/content/linedancing.mp4"
            },
            "input_streams": [],
            "output_streams": [
                {
                    "identifier": "c_ffmpeg_decoder_75_0",
                    "stream_alias": ""
                }
            ],
            "input_manager": "immediate",
            "scheduler": 0,
            "alias": "",
            "id": 75
        },
        {
            "module_info": {
                "name": "c_ffmpeg_filter",
                "type": "",
                "path": "",
                "entry": ""
            },
            "me

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/content/linedancing.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf56.4.101
  Duration: 00:00:12.00, start: 0.000000, bitrate: 359 kb/s
  Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 340x256 [SAR 256:255 DAR 4:3], 227 kb/s, 25 fps, 25 tbr, 12800 tbn, 50 tbc (default)
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
  Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 125 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
      vendor_id       : [0][0][0][0]


[2024-04-03 23:57:04.045] [info] c++ module constructed
[2024-04-03 23:57:04.045] [info] Constructing c++ module
[2024-04-03 23:57:04.045] [info] c++ module constructed
[2024-04-03 23:57:04.045] [info] Constructing c++ module
[2024-04-03 23:57:04.045] [info] c++ module constructed
[2024-04-03 23:57:04.046] [info] BMF Version: 0.0.9
[2024-04-03 23:57:04.046] [info] BMF Commit: e3c9730
[2024-04-03 23:57:04.046] [info] start init graph
[2024-04-03 23:57:04.046] [info] scheduler count2
debug queue size, node 75, queue size: 5
[2024-04-03 23:57:04.046] [info] node:c_ffmpeg_decoder 75 scheduler 0
debug queue size, node 77, queue size: 5
[2024-04-03 23:57:04.046] [info] node:c_ffmpeg_encoder 77 scheduler 1
debug queue size, node 76, queue size: 5
[2024-04-03 23:57:04.046] [info] Constructing c++ module
[2024-04-03 23:57:04.046] [info] c++ module constructed
[2024-04-03 23:57:04.046] [info] node:c_ffmpeg_filter 76 scheduler 0


Output #0, image2, to './extracted-images/simple_%03d.jpg':
  Metadata:
    encoder         : Lavf58.76.100
  Stream #0:0: Video: mjpeg, yuvj444p(pc), 340x256 [SAR 256:255 DAR 4:3], q=2-31, 2 fps, 2 tbr, 2 tbn, 2 tbc
[swscaler @ 0x7bc94408c600] deprecated pixel format used, make sure you did set range correctly


[2024-04-03 23:57:04.112] [info] node id:75 decode flushing
[2024-04-03 23:57:04.112] [info] node id:75 Process node end
[2024-04-03 23:57:04.114] [info] node id:75 close node
[2024-04-03 23:57:04.114] [info] node 75 close report, closed count: 1
[2024-04-03 23:57:04.114] [info] node id:76 eof received
[2024-04-03 23:57:04.114] [info] node id:76 eof processed, remove node from scheduler
[2024-04-03 23:57:04.114] [info] node id:76 process eof, add node to scheduler
[2024-04-03 23:57:04.114] [info] node id:76 Process node end
[2024-04-03 23:57:04.116] [info] node id:76 close node
[2024-04-03 23:57:04.116] [info] node 76 close report, closed count: 2
[2024-04-03 23:57:04.116] [info] node id:77 eof received
[2024-04-03 23:57:04.116] [info] node id:77 eof processed, remove node from scheduler
[2024-04-03 23:57:04.116] [info] node id:77 process eof, add node to scheduler
[2024-04-03 23:57:04.118] [info] node id:77 Process node end
[2024-04-03 23:57:04.118] [info] node id:77 close node
[2024-

 (repeated 19 more times)


Similar video: ./train/country_line_dancing/bTbC3w_NIvM.mp4
Similar video: ./train/country_line_dancing/zta-Iv-xK7I.mp4
Similar video: ./train/country_line_dancing/n2dWtEmNn5c.mp4


You can also use BMF for both decoding the query video and extracting frames. Look below!

In [ ]:
import os
import glob
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from pinecone import Pinecone
import bmf
import cv2
from IPython import display
from PIL import Image

PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_ENVIRONMENT = os.environ["PINECONE_ENVIRONMENT"]
PINECONE_DATABASE = os.environ["PINECONE_DATABASE"]

# Replace 'your_pinecone_api_key' with your actual Pinecone API key or use environment variables like I am here
pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)
index = pc.Index(PINECONE_DATABASE)

model = models.resnet18(pretrained=True)
model.eval()
#remove the last fully connectected layer
model = torch.nn.Sequential(*list(model.children())[:-1])

preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

input_video_path = "/content/linedancing.mp4"
output_path = "./extracted-images/simple_%03d.jpg"

graph = bmf.graph({'dump_graph':1})
video = graph.decode({
    "input_path": input_video_path,
}).fps(2)

try:
    (
        bmf.encode(
            video['video'],
            None,
            {
                "output_path": output_path,
                "format": "image2",
                "video_params": {"codec": "jpg"},
            }
        ).run()
    )
    print("Frame extraction completed successfully.")
except Exception as e:
    print(f"Error during frame extraction: {str(e)}")
# Load the extracted query frames and generate embeddings
query_frame_paths = glob.glob(output_path.replace("%03d", "*"))
query_embeddings = []

for frame_path in query_frame_paths:
    frame = cv2.imread(frame_path)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    frame = preprocess(frame)
    frame = frame.unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        embedding = model(frame)
        embedding = embedding.squeeze().numpy()

    query_embeddings.append(embedding)

query_embeddings = np.array(query_embeddings)
avg_query_embedding = np.mean(query_embeddings, axis=0)

# Perform similarity search using Pinecone
num_results = 5  # Number of similar videos to retrieve
results = index.query(
    vector=avg_query_embedding.tolist(),
    top_k=num_results,
    include_metadata=True,
    namespace='video_embeddings'
)

# Print the most similar video paths
for match in results['matches']:
    video_path = match['id']
    print(f"Similar video: {video_path}")

# Create a temporary directory to store the GIFs
tmp_dir = './tmp'
os.makedirs(tmp_dir, exist_ok=True)

def video_to_gif(video_path):
    gif_path = os.path.join(tmp_dir, video_path.split('/')[-1][:-4] + '.gif')
    frames = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(Image.fromarray(frame))
    cap.release()
    frames[0].save(fp=gif_path, format='GIF', append_images=frames[1:], save_all=True, loop=0)
    return gif_path

# Display the input video as a GIF
html = 'Query video "{}": <br/>'.format(input_video_path.split('/')[-1])
query_gif = video_to_gif(input_video_path)
html_line = '<img src="{}"> <br/>'.format(query_gif)
html += html_line
html += 'Top {} search results: <br/>'.format(num_results)

# Display the similar videos as GIFs
for path in [match['id'] for match in results['matches']]:
    gif_path = video_to_gif(path)
    html_line = '<img src="{}" style="display:inline;margin:1px"/>'.format(gif_path)
    html += html_line

display.HTML(html)

{
    "input_streams": [],
    "output_streams": [],
    "nodes": [
        {
            "module_info": {
                "name": "c_ffmpeg_decoder",
                "type": "",
                "path": "",
                "entry": ""
            },
            "meta_info": {
                "premodule_id": -1,
                "callback_binding": []
            },
            "option": {
                "input_path": "/content/linedancing.mp4"
            },
            "input_streams": [],
            "output_streams": [
                {
                    "identifier": "c_ffmpeg_decoder_81_0",
                    "stream_alias": ""
                }
            ],
            "input_manager": "immediate",
            "scheduler": 0,
            "alias": "",
            "id": 81
        },
        {
            "module_info": {
                "name": "c_ffmpeg_filter",
                "type": "",
                "path": "",
                "entry": ""
            },
            "me

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/content/linedancing.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf56.4.101
  Duration: 00:00:12.00, start: 0.000000, bitrate: 359 kb/s
  Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 340x256 [SAR 256:255 DAR 4:3], 227 kb/s, 25 fps, 25 tbr, 12800 tbn, 50 tbc (default)
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
  Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 125 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
      vendor_id       : [0][0][0][0]
Output #0, image2, to './extracted-images/simple_%03d.jpg':
  Metadata:
    encoder         : Lavf58.76.100
  Stream #0:0: Video: mjpeg, yuvj444p(pc), 340x256 [SAR 256:255 DAR 4:3], q=2-31, 2 fps, 2 tbr, 2 tbn, 2 tbc
[swscaler @ 0x7bc9481fb8c0] deprecated pixel format used, make sure you did set range

[2024-04-04 00:11:24.037] [info] node id:81 decode flushing
[2024-04-04 00:11:24.037] [info] node id:81 Process node end
[2024-04-04 00:11:24.039] [info] node id:81 close node
[2024-04-04 00:11:24.039] [info] node 81 close report, closed count: 1
[2024-04-04 00:11:24.039] [info] node id:82 eof received
[2024-04-04 00:11:24.039] [info] node id:82 eof processed, remove node from scheduler
[2024-04-04 00:11:24.039] [info] node id:82 process eof, add node to scheduler
[2024-04-04 00:11:24.039] [info] node id:82 Process node end
[2024-04-04 00:11:24.040] [info] node id:82 close node
[2024-04-04 00:11:24.040] [info] node 82 close report, closed count: 2
[2024-04-04 00:11:24.040] [info] node id:83 eof received
[2024-04-04 00:11:24.040] [info] node id:83 eof processed, remove node from scheduler
[2024-04-04 00:11:24.040] [info] node id:83 process eof, add node to scheduler
Frame extraction completed successfully.
[2024-04-04 00:11:24.042] [info] node id:83 Process node end
[2024-04-04 00:11:24